#### Simple Gen AI APP Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [4]:
loader=WebBaseLoader("https://en.wikipedia.org/wiki/Lionel_Messi")
loader

In [5]:
docs=loader.load()
## Print docs
## docs

In [6]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [7]:
documents[5]  ## Sixth chunk of document

Document(page_content='Edit links\n\n\n\n\n\n\n\n\n\n\n\nArticleTalk\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nReadView sourceView history\n\n\n\n\n\n\n\nTools\n\n\n\n\n\nTools\nmove to sidebar\nhide\n\n\n\n\t\tActions\n\t\n\n\nReadView sourceView history\n\n\n\n\n\n\t\tGeneral\n\t\n\n\nWhat links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code\n\n\n\n\n\n\t\tPrint/export\n\t\n\n\nDownload as PDFPrintable version\n\n\n\n\n\n\t\tIn other projects\n\t\n\n\nWikimedia CommonsWikinewsWikiquoteWikidata item\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\nmove to sidebar\nhide\n\n\n\n\n\n\n\n\n\n\n\n\nFrom Wikipedia, the free encyclopedia\n\n\n\nArgentine footballer (born 1987)\n"Messi" redirects here. For other uses, see Messi (disambiguation).', metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'})

In [33]:
## Creating Embeddings
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [34]:
## Store embeddings in FAISS Vector DB
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [35]:
vectorstoredb

In [36]:
## Query From a vector db when query text is not present in the documents stored in vector db
query="Narendra Modi"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Barcelona'

In [37]:
## Query From a vector db when query text is with the documents stored in vector db
query="Tell me about Lionel Messi's early career"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Club career\nMain article: Career of Lionel Messi §\xa0Club career\nBarcelona\n2004–2008: Rise to the first team\nMessi (pictured in 2005) at the age of 18 playing for FC Barcelona, where he played for 17 years\nMessi began the 2004–05 season as a guaranteed starter for the Barcelona B team, but after some lobbying by the senior players, he was promoted to the first team by manager Frank Rijkaard.[57] He made his La Liga debut for Barcelona on 16 October 2004 against Espanyol,[42] and scored his first senior goal on 1 May 2005 against Albacete, from an assist by Ronaldinho, becoming at that time the youngest-ever scorer for the club.[63][64] At 17 years, three months, and 22 days old, he was at the time the youngest player to represent Barcelona in an official competition, and the club won the league title during that season.[59][65]'

In [ ]:
## Create llm
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [39]:
## Create Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context, if you don't find any relevant information, say "I don't know".
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt) ## prompt here contains the context that is passed to the llm. 
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context, if you don\'t find any relevant information, say "I don\'t know".\n<context>\n{context}\n</context>\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x15764b5e0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x119697550>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [40]:
## Invoke Document Chain by giving single docmument in the prompt
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Was there any democratic backsliding in India under Narendra Modi?", ## prompt input is not present in the documents stored in vector db, it should say "I don't know" as instructed in the prompt.
    "context":[ Document(page_content="Messi made his competitive debut for Barcelona at age 17 in October 2004. He gradually established himself as an integral player for the club, and during his first uninterrupted season at age 22 in 2008–09 he helped Barcelona achieve the first treble in Spanish football. This resulted in Messi winning the first of four consecutive Ballon d'Ors, and by the 2011–12 season he set the European record for most goals in a season and established himself as Barcelona's all-time top scorer. During the 2014–15 campaign, where he became the all-time top scorer in La Liga, he led Barcelona to a historic second treble, leading to a fifth Ballon d'Or in 2015. He assumed Barcelona's captaincy in 2018 and won a record sixth Ballon d'Or in 2019. At Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four Champions Leagues, among others. Financial difficulties at Barcelona led to Messi signing with French club Paris Saint-Germain in August 2021, where he won the Ligue 1", metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'}),]
})

"I don't know."

In [ ]:
## Change the prompt template to include information in the responseif context is present
prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based on the provided context".
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm,prompt) 
document_chain
## Invoke Document Chain by giving single docmument in the prompt
document_chain.invoke({
    "input":"When did Messi make his competitive debut ", ## prompt input is present in the documents stored in vector db, it should return proper response.
    "context":[ Document(page_content="Messi made his competitive debut for Barcelona at age 17 in October 2004. He gradually established himself as an integral player for the club, and during his first uninterrupted season at age 22 in 2008–09 he helped Barcelona achieve the first treble in Spanish football. This resulted in Messi winning the first of four consecutive Ballon d'Ors, and by the 2011–12 season he set the European record for most goals in a season and established himself as Barcelona's all-time top scorer. During the 2014–15 campaign, where he became the all-time top scorer in La Liga, he led Barcelona to a historic second treble, leading to a fifth Ballon d'Or in 2015. He assumed Barcelona's captaincy in 2018 and won a record sixth Ballon d'Or in 2019. At Barcelona, Messi won a club-record 34 trophies, including ten La Liga titles and four Champions Leagues, among others. Financial difficulties at Barcelona led to Messi signing with French club Paris Saint-Germain in August 2021, where he won the Ligue 1", metadata={'source': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'title': 'Lionel Messi - Wikipedia', 'language': 'en'}),]
})

'When did Messi make his competitive debut for Barcelona?\n\nMessi made his competitive debut for Barcelona at age 17 in October 2004.'